In [1]:
import os
from dotenv import find_dotenv, load_dotenv

_ = load_dotenv(find_dotenv())

hf_token = os.getenv("HF_TOKEN")

In [2]:
from pathlib import Path
import sys

sys.path.append(Path(".").resolve().as_posix())

In [3]:
import torch
import json
from tqdm import tqdm

from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN
from llava.conversation import conv_templates, SeparatorStyle
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init
from llava.mm_utils import tokenizer_image_token, process_images, get_model_name_from_path

from PIL import Image
import requests
import math


def split_list(lst, n):
    """Split a list into n (roughly) equal-sized chunks"""
    chunk_size = math.ceil(len(lst) / n)  # integer division
    return [lst[i:i+chunk_size] for i in range(0, len(lst), chunk_size)]


def get_chunk(lst, n, k):
    chunks = split_list(lst, n)
    return chunks[k]


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(


[2024-04-16 06:35:01,447] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [4]:
# model_path = "liuhaotian/llava-v1.5-7b"
# model_path = "mistralai/Mistral-7B-v0.1"
model_path = "google/gemma-2b"
model_base = None
conv_mode = "gemma"
num_chunks = 1
chunk_idx = 0
temperature = 0.1
top_p = None
num_beams = 1

In [5]:
# # Model
# disable_torch_init()
# model_path = os.path.expanduser(model_path)
# model_name = get_model_name_from_path(model_path)
# tokenizer, model, image_processor, context_len = load_pretrained_model(model_path, model_base, model_name)

In [6]:
# Model but manually, lifted from load_pretrained_model

from transformers import AutoTokenizer
from llava.model import (
    LlavaConfig,
    LlavaMistralForCausalLM,
    LlavaLlamaForCausalLM,
    LlavaGemmaForCausalLM,
)
from llava.constants import DEFAULT_IMAGE_PATCH_TOKEN

disable_torch_init()
model_path = os.path.expanduser(model_path)
model_name = get_model_name_from_path(model_path)

device_map = "auto"

kwargs = {
    "device_map": device_map,
    "torch_dtype": torch.float16,
}

tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)

model = LlavaGemmaForCausalLM.from_pretrained(
    model_path, low_cpu_mem_usage=True, **kwargs
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
# Vision tower with pretrain

from llava.train.train import (
    DataArguments,
    ModelArguments,
    TrainingArguments,
)

model_args = ModelArguments(
    vision_tower="LanguageBind/LanguageBind_Video_merge",
    # pretrain_mm_mlp_adapter="mm_projector.bin",
    mm_vision_select_layer=-2,
    mm_use_im_start_end=False,
    mm_use_im_patch_token=False,
)

data_args = DataArguments()
training_args = TrainingArguments(
    output_dir="./llava_output",
)

In [8]:
model.get_model().initialize_vision_modules(
    model_args=model_args,
    fsdp=training_args.fsdp
)

vision_tower = model.get_vision_tower()
vision_tower.to(dtype=torch.bfloat16 if training_args.bf16 else torch.float16, device=training_args.device)

data_args.image_processor = vision_tower.video_processor
data_args.is_multimodal = True

model.config.image_aspect_ratio = data_args.image_aspect_ratio
model.config.tokenizer_padding_side = tokenizer.padding_side
model.config.tokenizer_model_max_length = tokenizer.model_max_length

model.config.tune_mm_mlp_adapter = training_args.tune_mm_mlp_adapter = model_args.tune_mm_mlp_adapter
if model_args.tune_mm_mlp_adapter:
    model.requires_grad_(False)
    for p in model.get_model().mm_projector.parameters():
        p.requires_grad = True

model.config.freeze_mm_mlp_adapter = training_args.freeze_mm_mlp_adapter
if training_args.freeze_mm_mlp_adapter:
    for p in model.get_model().mm_projector.parameters():
        p.requires_grad = False

model.config.mm_use_im_start_end = data_args.mm_use_im_start_end = model_args.mm_use_im_start_end
model.config.mm_projector_lr = training_args.mm_projector_lr
training_args.use_im_start_end = model_args.mm_use_im_start_end
model.config.mm_use_im_patch_token = model_args.mm_use_im_patch_token
model.initialize_vision_tokenizer(model_args, tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:803: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [9]:
image_processor = None

mm_use_im_start_end = getattr(model.config, "mm_use_im_start_end", False)
mm_use_im_patch_token = getattr(model.config, "mm_use_im_patch_token", True)

if mm_use_im_patch_token:
    tokenizer.add_tokens([DEFAULT_IMAGE_PATCH_TOKEN], special_tokens=True)
if mm_use_im_start_end:
    tokenizer.add_tokens(
        [DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN], special_tokens=True
    )
model.resize_token_embeddings(len(tokenizer))

vision_tower = model.get_vision_tower()
if not vision_tower.is_loaded:
    vision_tower.load_model(device_map=device_map)
if device_map != "auto":
    vision_tower.to(device=device_map, dtype=torch.float16)
image_processor = vision_tower.video_processor

if hasattr(model.config, "max_sequence_length"):
    context_len = model.config.max_sequence_length
else:
    context_len = 2048

In [10]:
model.get_vision_tower().hidden_size

1024

In [11]:
model.get_model().config

LlavaGemmaConfig {
  "_name_or_path": "google/gemma-2b",
  "architectures": [
    "GemmaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "eos_token_id": 1,
  "freeze_mm_mlp_adapter": false,
  "head_dim": 256,
  "hidden_act": "gelu",
  "hidden_activation": null,
  "hidden_size": 2048,
  "image_aspect_ratio": "square",
  "initializer_range": 0.02,
  "intermediate_size": 16384,
  "max_position_embeddings": 8192,
  "mm_hidden_size": 1024,
  "mm_patch_merge_type": "flat",
  "mm_projector_lr": null,
  "mm_projector_type": "linear",
  "mm_use_im_patch_token": false,
  "mm_use_im_start_end": false,
  "mm_vision_select_feature": "patch",
  "mm_vision_select_layer": -2,
  "mm_vision_tower": "LanguageBind/LanguageBind_Video_merge",
  "model_type": "llava_gemma",
  "num_attention_heads": 8,
  "num_hidden_layers": 18,
  "num_key_value_heads": 1,
  "pad_token_id": 0,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tokenizer_

In [12]:
qs = "What is unusual about this image?"
cur_prompt = qs

# Insert special image tokens into the text prompt

if model.config.mm_use_im_start_end:
    image_tokens = " ".join([DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_TOKEN + DEFAULT_IM_END_TOKEN] * model.get_vision_tower().config.num_frames) + "\n"
else:
    image_tokens = " ".join([DEFAULT_IMAGE_TOKEN] * model.get_vision_tower().config.num_frames) + "\n"
qs = image_tokens + qs

# Construct conversation prompt

conv = conv_templates[conv_mode].copy()
conv.append_message(conv.roles[0], qs)
conv.append_message(conv.roles[1], None)
prompt = conv.get_prompt()

input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).cuda()

# url = "https://www.ilankelman.org/stopsigns/australia.jpg"
# image = Image.open(requests.get(url, stream=True).raw)
# image_tensor = process_images([image], image_processor, model.config)[0]

video = "/data/vlm_sandbox/videos/lie1.mp4"
video_tensor = image_processor(video, return_tensors='pt')['pixel_values'].to("cuda")
model.get_model().mm_projector.to("cuda", dtype=torch.float16)

with torch.inference_mode():
    output_ids = model.generate(
        input_ids,
        images=video_tensor.half().cuda(),
        # image_sizes=[image.size],
        do_sample=True if temperature > 0 else False,
        temperature=temperature,
        top_p=top_p,
        num_beams=num_beams,
        # no_repeat_ngram_size=3,
        max_new_tokens=1024,
        use_cache=True)

outputs = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0].strip()

torch.Size([1, 3, 8, 224, 224])
torch.Size([1, 8, 257, 1024])
image_features shape after projection and concat 8
torch.Size([257, 2048])


RuntimeError: probability tensor contains either `inf`, `nan` or element < 0